In [39]:
import subprocess
import time
import datetime
import collections
import numpy as np
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor

In [163]:
# 長めな処理
def gcd(pair):
    a, b = pair
    if a < b:
        low = a
    else:
        low = b
    # print("a: %f b: %f" %(a, b))
    for i in range(int(low), 0, -1):
        if a % i == 0 and b % i ==0:
            return i

# 逐次処理
def single_thread_test(sample_func, arr_list, process_num_list):
    result_obj = {}
    for process_num in process_num_list:
        results = []
        for i in range(process_num):
            start = time.time()
            results = list(map(sumple_func, arr_list))
            end = time.time()
            results.append(end - start)
        result_obj[process_num] = results
    return result_obj


# ThreadPoolExecutor
def thread_pool_test(sample_func, arr_list, process_num_list):
    result_obj = {}
    for process_num in process_num_list:
        results = []
        for i in range(process_num):
            start = time.time()
            pool = ThreadPoolExecutor(max_workers=2)
            results = list(pool.map(sample_func, arr_list))
            end = time.time()
            results.append(end - start)
        result_obj[process_num] = results
    return result_obj


# ProcessPoolExecutor
def process_pool_test(sample_func, arr_list, procews_num_list):
    result_obj = {}
    for process_num in process_num_list:
        results = []
        for i in range(process_num):
            start = time.time()
            pool = ProcessPoolExecutor(max_workers=2)
            results = list(pool.map(sample_func, arr_list))
            end = time.time()
            results.append(end - start)
        result_obj[process_num] = results
    return result_obj

In [160]:
# 処理の個数
process_num_list = [5, 10, 100, 1000, 10000]
# 処理の重さ
process_weight_list = [100, 10000, 1000000]

# 検証用データオブジェクト
arr_obj = {}
for process_num in process_num_list:
    arr_obj[process_num] = {}
    for process_weight in process_weight_list:
        arr_obj[process_num][process_weight] = {}
        for index in range(process_num):
            arr_list = []
            for value in range(10):
                arr_list.append(np.random.rand(2) * process_weight)
            arr_obj[process_num][process_weight][index] = arr_list

In [164]:
# 検証の実行
